<a href="https://colab.research.google.com/github/denis911/image-classification-transfer-learning-pytorch-keras-comparison/blob/main/HW8_hair_type_classification_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download the dataset

In [1]:
!nvidia-smi # nvidia system information for T4 Tesla GPU

Mon Dec  1 22:00:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# We'll build a model for classifying various hair types. For this, we will use the Hair Type dataset that was obtained from Kaggle and slightly rebuilt.

# RUN ONCE:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip > /dev/null

--2025-12-01 22:00:25--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-01T22%3A41%3A46Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-01T21%3A41%3A04Z&ske=2025-12-01T22%3A41%3A46Z&sks=b&skv=2018-11-09&sig=epc9sxzBNn83KWworHsmhM3HzKfR0uyPp7F0Te4hDDA%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDYyODIyNSwibmJmIjoxNzY0NjI2NDI1LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG

In [3]:
# RUN ONCE
# unzip dataset into Colab home folder and check:
!unzip data.zip > /dev/null
# > /dev/null to prohibit long output, then check files manually

In [4]:
# Reproducibility
# Reproducibility in deep learning is a multifaceted challenge that requires attention to both software and hardware details. In some cases, we can't guarantee exactly the same results during the same experiment runs.

# Therefore we suggest to set the random number seed generators by:

import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



In [5]:
# STEP 0 - Custom Dataset Class
# First, create a PyTorch Dataset to load images:

import os
from torch.utils.data import Dataset
from PIL import Image

class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label



## Model building

In [6]:
# Model to classify curly vs straight hair from the photo
# We will use Convolutional Neural Network (CNN) and PyTorch.

# TASK - develop the model with following structure:

# The shape for input should be (3, 200, 200) (channels first format in PyTorch)
# Next, create a convolutional layer (nn.Conv2d):
# Use 32 filters (output channels)
# Kernel size should be (3, 3) (that's the size of the filter), padding = 0, stride = 1
# Use 'relu' as activation
# Reduce the size of the feature map with max pooling (nn.MaxPool2d)
# Set the pooling size to (2, 2)
# Turn the multi-dimensional result into vectors using flatten or view
# Next, add a nn.Linear layer with 64 neurons and 'relu' activation
# Finally, create the nn.Linear layer with 1 neuron - this will be the output
# The output layer should have an activation - use the appropriate activation for the binary classification case - SIGMOID

# As optimizer use torch.optim.SGD with the following parameters:

# torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

import torch
import torch.nn as nn
import torch.nn.functional as F

class HairClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # Input: (3, 200, 200)

        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=0
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # After conv1: (32, 198, 198)
        # After pool: (32, 99, 99)

        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)     # <-- raw logits, no sigmoid
        # our loss will be in the training nn.BCEWithLogitsLoss()

        return x


# Q1 - we use loss  nn.BCEWithLogitsLoss()


In [7]:
# Q2 = Total params: 20,073,473
# What's the total number of parameters of the model?
# In PyTorch, you can find the total number of parameters using:

# Option 1: Using torchsummary (install with: pip install torchsummary)
# from torchsummary import summary
# model = HairClassifier()
# summary(model, (3, 200, 200))

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

## Generators and Training

In [ ]:
# STEP 1 - use the following transformation for both train and test sets:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [ ]:
# STEP 2 - Create DataLoaders

from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir='/content/data/train',
    transform=train_transforms
)

val_dataset = HairDataset(
    data_dir='/content/data/test',
    transform=train_transforms
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# STEP 3 - train the Model

import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# We don't need to do any additional pre-processing for the images.
# Use batch_size=20
# Use shuffle=True for both training, but False for test.

# Now fit the model.
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

# Question 3
# What is the median of training accuracy for all the epochs for this model?
# 0.05
# 0.12
# 0.40
# 0.84

# Question 4
# What is the standard deviation of training loss for all the epochs for this model?
# 0.007
# 0.078
# 0.171
# 1.710
